# **Working with Astra DB's SAI Analyzers and Vector Search**

---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/msmygit/dse-tidbits/blob/master/documents/astra_vector_search_with_sai_analyzer/Vector_Search_with_AstraDB_SAI_Analyzers.ipynb)

This hands-on tutorial will show you how you can add Astra DB's SAI analyzer features to power your own Generative AI applications with just a few lines of code.

We will build together a sample end-to-end use case that will be able to understand and respond to human language queries about the relational data stored in your PostgreSQL database. In fact, we will further push the creative limits of the application by teaching it to generate new content based on our existing dataset.

## Prerequisites

[Create a serverless database with Astra Vector Search and set up its schema](https://docs.datastax.com/en/astra-serverless/docs/vector-search/chatbot-quickstart.html#_prerequisites). For the reminder of this notebook, we will be using the keyspace as `baselines` and the table name as `clients` so, if you want to simply copy paste, create it the same.

## Objective

After completing the steps in this notebook:
- You will have a good understanding of how to use the [SAI analyzers](https://docs.datastax.com/en/astra-serverless/docs/vector-search/using-analyzers.html) along with your vector embeddings in Astra DB. Learn more about [vector search in Astra DB](https://docs.datastax.com/en/astra-serverless/docs/vector-search/overview.html).
- You will get a hands-on experience with using the SAI analyzers and will also have other resources to expand on.
- _Storage-Attached Index (SAI)_ is the one that is already powering Vector Search experience within Astra DB today and these same capabilities will be contributed to Apache Cassandra® via [CEP-7](https://cwiki.apache.org/confluence/x/7DZ4CQ) & [CEP-30](https://cwiki.apache.org/confluence/x/OQ40Dw) enhancement proposals.

There are [many advantages to leveraging SAI](https://docs.datastax.com/en/cql/astra/docs/developing/indexing/sai/sai-concepts.html), but in particular, SAI shares common index data across multiple indexes on the same table. This unique feature gives users the ability to create many more indexes without running into scalability issues.

The following charts give an indication of the space saving advantage of using SAI vs alternatives using a financial time series data model,

![Alt text](./sai_disk_efficiency.png)

## Using this interactive notebook

Click the **run** icon on the top left corner ▶️  of each cell within this notebook.

> 💡 Alternatively, you can run the currently selected cell with `Ctrl + Enter` (or `⌘ + Enter` on a Mac).

> ⚠️ **To avoid any errors**, wait for each cell to finish in their order before clicking the next “run” icon.

## Prepare the table schema

- Naviate to [`CQL Console`](https://docs.datastax.com/en/astra-serverless/docs/connect/cql/connect-cqlsh.html) within the Astra portal.
- Type in `USE baselines;` and hit enter key.
- Create the table using the below command,

   ```
   CREATE TABLE IF NOT EXISTS baselines.clients (
       uniqueid uuid primary key,
       firstname text,
       lastname text,
       birthday date,
       nextappt timestamp,
       newpatient boolean,
       photo text,
       traits vector<float, 3>
   );
   CREATE CUSTOM INDEX baselines_clients_birthday_idx ON baselines.clients (birthday) USING 'StorageAttachedIndex';
   CREATE CUSTOM INDEX baselines_clients_lastname_idx ON baselines.clients (lastname) USING 'StorageAttachedIndex';
   CREATE CUSTOM INDEX baselines_clients_traits_vector_idx ON baselines.clients (traits) USING 'StorageAttachedIndex' WITH OPTIONS = {'similarity_function': 'dot_product'};
   ```
- Verify the table's schema by running `DESC TABLE clients;` command in the console.  

## Loading the test data

Here, we will use a sample data that is prepared. The normalized (i.e. ranging from `-1` to `1`) vector embedding could be generated by leveraging services such as OpenAI, etc. Be sure to match the vector dimension in the table's column (i.e. `traits` here in our case) to the model being used. For simplicity sake, we will use the vector embedding which has a dimension of `3`.

- Copy & Run the following insert statements within the Astra's CQL Console:
    <details>
       <summary>Expand/Collapse to view the insert statements</summary>
       
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (D85745B1-4BEC-43D7-8B77-DD164CB9D1B8, 'Alice', 'Apple', '1984-01-24', '2020-10-20 12:00:00', true, 'imageurl1', [0.853663685, -0.370721894, -0.838156652]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (2A4F139F-0BBF-4A6F-B982-5400F11D2F2B, 'Zeke', 'Apple', '1961-12-30', '2020-10-20 12:30:00', false, 'imageurl2', [-0.2386359, 0.061663079, 0.159176746]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (DF649261-89CB-446B-9998-FFA2D17506F9, 'Lorenzo', 'Banana', '1963-09-03', '2020-10-20 13:00:00', false, 'imageurl3', [0.29688748, 0.162187505, -0.899572388]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (808E6BBF-A0F4-4E4C-9C97-E36751D51A8B, 'Miley', 'Banana', '1969-02-06', '2020-10-20 13:30:00', false, 'imageurl4', [ 0.71793891, 0.304702581, -0.319685541]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (3D458A4D-2F54-4271-BEDC-1FC316B3CC96, 'Cheryl', 'Banana', '1970-07-11', '2020-10-20 14:00:00', false, 'imageurl5', [-0.828286076, 0.951830234, 0.052958224]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (287AB6B4-1AA6-45DF-B6F8-2BE253B9AACE, 'Red', 'Currant', '1974-02-18', '2020-10-20 15:00:00', false, 'imageurl6', [-0.503873582, 0.818124782, 0.481385136]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (AB49D151-CC04-40DC-AEEA-0A4E5F59D69A, 'Matthew', 'Durian', '1976-11-11', '2020-10-19 12:30:00', false, 'imageurl7', [-0.361109664, -0.379330015, -0.725305926]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (783CE790-16B4-4645-B27C-4FDF3994A755, 'Vanessa', 'Elderberry', '1977-12-03', '2020-10-20 15:30:00', false, 'imageurl8', [0.784867341, 0.025673095, 0.828155797]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (D23997E4-CCCB-46BB-B92F-0D4582A68809, 'Elaine', 'Elderberry', '1979-11-16', '2020-10-20 10:00:00', true, 'imageurl9', [-0.241611772, -0.344717538, -0.764725406]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (36C386C1-3C3B-49FC-81B1-391D5537453D, 'Phoebe', 'Fig', '1986-01-27', '2020-10-21 11:00:00', false, 'imageurl10', [0.823133083, 0.532633194, -0.829494313]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (00FEE7EE-8F93-4C2E-A8BE-3ADD81235822, 'Patricia', 'Grape', '1986-06-24', '2020-10-21 12:00:00', false, 'imageurl11', [-0.788176132, 0.649411352, 0.930390198]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (B9DB7E99-AD1C-49B1-97C6-87154663AEF4, 'Herb', 'Huckleberry', '1990-07-09', '2020-10-21 13:00:00', false, 'imageurl12', [0.498928801, -0.835162113, -0.916987826]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (F4DB7673-CA4E-4382-BDCD-2C1704363590, 'John-Henry', 'Huckleberry', '1979-11-16', '2020-10-21 14:00:00', false, 'imageurl13', [-0.609193496, -0.077998017,     -0.       796146016]);
         
         INSERT INTO clients (uniqueid, firstname, lastname, birthday, nextappt, newpatient, photo, traits) 
         VALUES (F4DB7673-CA4E-4382-BDCD-2C1704363595, 'Sven', 'Åskådare', '1967-11-07', '2020-10-21 14:00:00', false, 'imageurl15', [-0.97499609, -0.981892464, -0.671463954]);
    
    </details>

- Verify by running the following command:
    ```
    token@cqlsh:baselines> SELECT * FROM clients ;
    
     uniqueid                             | birthday   | firstname  | lastname    | newpatient | nextappt                        | photo      | traits
    --------------------------------------+------------+------------+-------------+------------+---------------------------------+------------+-----------------------------------
     36c386c1-3c3b-49fc-81b1-391d5537453d | 1986-01-27 |     Phoebe |         Fig |      False | 2020-10-21 11:00:00.000000+0000 | imageurl10 |   [0.823133, 0.532633, -0.829494]
     df649261-89cb-446b-9998-ffa2d17506f9 | 1963-09-03 |    Lorenzo |      Banana |      False | 2020-10-20 13:00:00.000000+0000 |  imageurl3 |   [0.296887, 0.162188, -0.899572]
     287ab6b4-1aa6-45df-b6f8-2be253b9aace | 1974-02-18 |        Red |     Currant |      False | 2020-10-20 15:00:00.000000+0000 |  imageurl6 |   [-0.503874, 0.818125, 0.481385]
     808e6bbf-a0f4-4e4c-9c97-e36751d51a8b | 1969-02-06 |      Miley |      Banana |      False | 2020-10-20 13:30:00.000000+0000 |  imageurl4 |   [0.717939, 0.304703, -0.319686]
     3d458a4d-2f54-4271-bedc-1fc316b3cc96 | 1970-07-11 |     Cheryl |      Banana |      False | 2020-10-20 14:00:00.000000+0000 |  imageurl5 |    [-0.828286, 0.95183, 0.052958]
     f4db7673-ca4e-4382-bdcd-2c1704363590 | 1979-11-16 | John-Henry | Huckleberry |      False | 2020-10-21 14:00:00.000000+0000 | imageurl13 | [-0.609194, -0.077998, -0.796146]
     00fee7ee-8f93-4c2e-a8be-3add81235822 | 1986-06-24 |   Patricia |       Grape |      False | 2020-10-21 12:00:00.000000+0000 | imageurl11 |    [-0.788176, 0.649411, 0.93039]
     b9db7e99-ad1c-49b1-97c6-87154663aef4 | 1990-07-09 |       Herb | Huckleberry |      False | 2020-10-21 13:00:00.000000+0000 | imageurl12 |  [0.498929, -0.835162, -0.916988]
     ab49d151-cc04-40dc-aeea-0a4e5f59d69a | 1976-11-11 |    Matthew |      Durian |      False | 2020-10-19 12:30:00.000000+0000 |  imageurl7 |   [-0.36111, -0.37933, -0.725306]
     d23997e4-cccb-46bb-b92f-0d4582a68809 | 1979-11-16 |     Elaine |  Elderberry |       True | 2020-10-20 10:00:00.000000+0000 |  imageurl9 | [-0.241612, -0.344718, -0.764725]
     f4db7673-ca4e-4382-bdcd-2c1704363595 | 1967-11-07 |       Sven |    Åskådare |      False | 2020-10-21 14:00:00.000000+0000 | imageurl15 | [-0.974996, -0.981892, -0.671464]
     2a4f139f-0bbf-4a6f-b982-5400f11d2f2b | 1961-12-30 |       Zeke |       Apple |      False | 2020-10-20 12:30:00.000000+0000 |  imageurl2 |   [-0.238636, 0.061663, 0.159177]
     d85745b1-4bec-43d7-8b77-dd164cb9d1b8 | 1984-01-24 |      Alice |       Apple |       True | 2020-10-20 12:00:00.000000+0000 |  imageurl1 |  [0.853664, -0.370722, -0.838157]
     783ce790-16b4-4645-b27c-4fdf3994a755 | 1977-12-03 |    Vanessa |  Elderberry |      False | 2020-10-20 15:30:00.000000+0000 |  imageurl8 |    [0.784867, 0.025673, 0.828156]
    
    (14 rows)
    ```

## Leveraging SAI in the read queries

- All the below examples will be using index based reads without involving the traditional primary key approach.
- Now that we've all the data and indices set, let's start reading back the data. Let's try to use basic index first,

  ```
  token@cqlsh:baselines> SELECT firstname FROM baselines.clients WHERE lastname = 'Apple';
  
   firstname
  -----------
        Zeke
       Alice
  
  (2 rows)
  ```
- Let's read the desired data back using the vector index, i.e. let's perform a vector search:

  ```
  token@cqlsh:baselines> SELECT firstname FROM baselines.clients ORDER BY traits ANN OF [0.5,0.5,0.5] LIMIT 5;
  
   firstname | traits
  -----------+---------------------------------
     Vanessa |  [0.784867, 0.025673, 0.828156]
         Red | [-0.503874, 0.818125, 0.481385]
    Patricia |  [-0.788176, 0.649411, 0.93039]
       Miley | [0.717939, 0.304703, -0.319686]
      Phoebe | [0.823133, 0.532633, -0.829494]
  
  (5 rows)
  ```

- Now, let's combine the regular index with vector search:

  ```
  token@cqlsh:baselines> SELECT firstname, traits FROM baselines.clients WHERE lastname = 'Apple' ORDER BY traits ANN OF [0.5,0.5,0.5] LIMIT 5;
  token@cqlsh:baselines> SELECT * FROM baselines.clients WHERE lastname = 'Apple';
  
   uniqueid                             | birthday   | firstname  | lastname    | newpatient | nextappt                        | photo      | traits
  --------------------------------------+------------+------------+-------------+------------+---------------------------------+------------+-----------------------------------
   2a4f139f-0bbf-4a6f-b982-5400f11d2f2b | 1961-12-30 |       Zeke |       Apple |      False | 2020-10-20 12:30:00.000000+0000 |  imageurl2 |   [-0.238636, 0.061663, 0.159177]
   d85745b1-4bec-43d7-8b77-dd164cb9d1b8 | 1984-01-24 |      Alice |       Apple |       True | 2020-10-20 12:00:00.000000+0000 |  imageurl1 |  [0.853664, -0.370722, -0.838157]
  
  (2 rows)
  ```

- We could do powerful composing of regular indicies with vector search as follows too:

  ```
  token@cqlsh:baselines> token@cqlsh:baselines> SELECT * FROM baselines.clients WHERE birthday > '1990-01-01' OR lastname = 'Fig' ORDER BY traits ANN OF [0.5,0.5,0.5] LIMIT 5;
  
   uniqueid                             | birthday   | firstname | lastname    | newpatient | nextappt                        | photo      | traits
  --------------------------------------+------------+-----------+-------------+------------+---------------------------------+------------+----------------------------------
   36c386c1-3c3b-49fc-81b1-391d5537453d | 1986-01-27 |    Phoebe |         Fig |      False | 2020-10-21 11:00:00.000000+0000 | imageurl10 |  [0.823133, 0.532633, -0.829494]
  
  (1 rows)
  ```

## Leveraging Analyzers with vector search on the reads

Using analyzers with vector search allows you to create a hybrid search that includes your keyword and a vector embedding. Instead of returning only a list of results, a large language model (LLM) can return similar results based on the analyzer, which could be a brand or size or in our example a particular trait.

For example, if you ask an LLM “Tell me about available shoes” and this query is done with vector search against your Astra DB table, you would get a list of several shoes with a variety of features.

- We will drop the existing index on `lastname` column by running `DROP INDEX baselines_clients_lastname_idx;`

- We can create an SAI index using the new index_analyzer option.  Usually standard is a good starting point:

  ```
  CREATE CUSTOM INDEX baselines_clients_lastname_idx_analyzer ON baselines.clients (lastname) USING 'StorageAttachedIndex' WITH OPTIONS = { 'index_analyzer': 'standard'};
  ```

- And you can query like this to get both rows.  Note how the analyzer takes care of splitting the text up into case-independent terms. Notice our input with all lowercase `banana`:

  ```
  token@cqlsh:baselines> SELECT firstname, lastname FROM baselines.clients WHERE lastname : 'banana';
  
   firstname | lastname
  -----------+----------
     Lorenzo |   Banana
       Miley |   Banana
      Cheryl |   Banana
  
  (3 rows)
  ```

- Now comes the powerful part of combining the analyzers with the vector search. You get the approximate nearest neighbor of the relevant _traits_ plus you could combine to further filter result to your custom liking, let's say the clients with lastname of `banana`:

  ```
  token@cqlsh:baselines> SELECT firstname, lastname FROM baselines.clients WHERE lastname : 'banana' ORDER BY traits ANN OF [0.29, 0.95, 0.05] LIMIT 5;
  
   firstname | lastname | traits
  -----------+----------+---------------------------------
      Cheryl |   Banana |  [-0.828286, 0.95183, 0.052958]
       Miley |   Banana | [0.717939, 0.304703, -0.319686]
     Lorenzo |   Banana | [0.296887, 0.162188, -0.899572]
  
  (3 rows)
  ```
- Points to be noted in the earlier query are:
  - It automatically matches the rows with lastname `Bananas` (case-insensitive).
  - And, it filters the `5` vector search matches further down with the analyzer output.

- Now, what if we want to search using a stemmed word like `Bananas`? This below query would yield no results because standard analyzer does not include [stemming](https://en.wikipedia.org/wiki/Stemming):

  ```
  token@cqlsh:baselines> select * from clients where lastname : 'Bananas';
  
   uniqueid | birthday | firstname | lastname | newpatient | nextappt | photo | traits
  ----------+----------+-----------+----------+------------+----------+-------+--------
  
  (0 rows)
  ```

- This is where custom configuration of the analyzer comes into play. There is one primary option to customize for now. There could be more coming in future.
  - `index_analyzer`: configures how to analyze the column’s values before indexing them. Note that this analyzer is applied to the query’s term search as well.

- We will drop the existing index on `lastname` column by running `DROP INDEX baselines_clients_lastname_idx_analyzer;` and re-create it as follows to enable [stemming](https://en.wikipedia.org/wiki/Stemming):

  ```
  CREATE CUSTOM INDEX baselines_clients_lastname_idx_analyzer ON baselines.clients (lastname) USING 'StorageAttachedIndex' WITH OPTIONS = {'index_analyzer': '{ "tokenizer" :   {"name" : "standard"}, "filters" : [{"name" : "porterstem"}] }'};

  token@cqlsh:baselines> SELECT firstname, lastname, traits FROM baselines.clients WHERE lastname : 'Bananas' ORDER BY traits ANN OF [0.29, 0.95, 0.05] LIMIT 5;
  
   firstname | lastname | traits
  -----------+----------+---------------------------------
      Cheryl |   Banana |  [-0.828286, 0.95183, 0.052958]
       Miley |   Banana | [0.717939, 0.304703, -0.319686]
     Lorenzo |   Banana | [0.296887, 0.162188, -0.899572]
  
  (3 rows)
  ```

- Refer to [this documentation](https://docs.datastax.com/en/astra-serverless/docs/vector-search/using-analyzers.html) for other power user configuration options.

### (Optional) More complex example using an `edgeNGram` tokenizer
Tokenizes the input from an edge into n-grams of given size(s). This `Tokenizer` create n-grams from the beginning edge of a input token.

Index creation in this case will be as follows:
```
CREATE CUSTOM INDEX baselines_clients_firstname_idex_analyzer ON baselines.clients (firstname) USING 'StorageAttachedIndex' WITH OPTIONS = {
  'index_analyzer': '{
    "tokenizer" : {
      "name" : "edgeNGram",
      "args" : {
            "minGramSize":"1",
            "maxGramSize":"3"
            }
    },
    "filters" : [
      {
        "name" : "lowercase",
        "args": {}
      }
    ],
    "charFilters" : []
  }'
};
```

The following queries will now be possible unlocking even more use cases:
- `SELECT * FROM baselines.clients WHERE firstname : 'ph';`
- `SELECT * FROM baselines.clients WHERE firstname : 'phOEbE';`
- `SELECT * FROM baselines.clients WHERE firstname : 'phOEbE' ORDER BY traits ANN OF [0.29, 0.95, 0.05] LIMIT 1;`

Results will be:
```
token@cqlsh:baselines> SELECT * FROM baselines.clients WHERE firstname : 'phOEbE' ORDER BY traits ANN OF [0.29, 0.95, 0.05] LIMIT 1;

 uniqueid                             | birthday   | firstname | lastname | newpatient | nextappt                        | photo      | traits
--------------------------------------+------------+-----------+----------+------------+---------------------------------+------------+---------------------------------
 36c386c1-3c3b-49fc-81b1-391d5537453d | 1986-01-27 |    Phoebe |      Fig |      False | 2020-10-21 11:00:00.000000+0000 | imageurl10 | [0.823133, 0.532633, -0.829494]

(1 rows)
```

##  (Optional) Cleaning up
When you no longer need your database along with the data, you could [terminate your database](https://docs.datastax.com/en/astra-serverless/docs/manage/db/manage-terminate-db.html).

Generative AI is a powerful paradigm shift in application development that lets you create novel applications to serve users in new ways - [from answering patients' complex medical questions](https://cloud.google.com/blog/topics/healthcare-life-sciences/sharing-google-med-palm-2-medical-large-language-model) to [helping enterprises analyze cyberattacks](https://cloud.google.com/blog/products/identity-security/rsa-google-cloud-security-ai-workbench-generative-ai). In this demo, we showed you just a couple examples of powerful Astra features that you can leverage further to create powerful experiences by combining LLMs, Agents, [**LangStream**](https://docs.langstream.ai) and Astra platform.

We can't wait to see what you build with it! 🚀

---